In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pygame
import sys

pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [71]:
# Genrate Medications
meds_df = pd.DataFrame({
    'Medicine': [],
    'Lifespan': []
})
max_meds = 10
lifespan_mean = 25
lifespan_std = 10

LIFESPAN_MIN = 5

medicine = set(['{:04x}'.format(np.random.randint(0, 0xFFFF)) for _ in range(max_meds)])
# ensures there are max_meds unique values
while len(medicine) < max_meds: medicine.add('{:04x}'.format(np.random.randint(0, 0xFFFF)))
medicine = list(medicine)

lifespan = np.random.normal(lifespan_mean, lifespan_std, max_meds)
lifespan = np.round(np.maximum(lifespan, LIFESPAN_MIN))

meds_df['Medicine'] = medicine
meds_df['Lifespan'] = lifespan

meds_df.head()


Medicine  Lifespan
0     d89b       6.0
1     2190      19.0
2     81f6      17.0
3     4edd      34.0
4     e893      24.0

In [74]:
# Generate Clients
client_count = 100
MAXLAT = 256
MAXLONG = 256
MAX_DOSAGE = 3

latitudes = np.random.randint(0, MAXLAT, client_count)
longitudes = np.random.randint(0, MAXLONG, client_count)

cs_df = pd.DataFrame({
    'Client': ['Client {:03d}'.format(i) for i in range(client_count)],
    'Lat': latitudes,
    'Long': longitudes,
    'Medicine': np.random.choice(medicine, client_count),
    'Dosage': np.random.randint(1, MAX_DOSAGE, client_count)
})
# cs_df.index.name = 'Client'
cs_df.head()

Client  Lat  Long Medicine  Dosage
0  Client 000   73    95     c32e       2
1  Client 001  118   125     9751       2
2  Client 002  240   233     e893       2
3  Client 003  198   228     2692       1
4  Client 004    0    21     2692       1

In [75]:
# Generate Pharmacies
pharmacy_count = 10
pharmacy_names = [f'pharm{i}' for i in range(pharmacy_count)]

latitudes = np.random.randint(0, MAXLAT, pharmacy_count)
longitudes = np.random.randint(0, MAXLONG, pharmacy_count)

rx_df = pd.DataFrame({
    'Name': pharmacy_names,
    'Lat': latitudes,
    'Long': longitudes
})

In [76]:
# Assign Customers to Pharmacies
# client are assigned to closest pharm

# distance between each client and each pharm
client_coords = cs_df[['Lat', 'Long']].values
pharm = np.column_stack((latitudes[:pharmacy_count], longitudes[:pharmacy_count]))

distances = np.sqrt(
    (
        (client_coords[:, np.newaxis, :] - pharm[np.newaxis, :, :]) ** 2
    ).sum(axis=2))


# Assign each customer to the closest pharmacy
closest_pharmacies = np.argmin(distances, axis=1)
cs_df['Assigned_Pharmacy'] = closest_pharmacies

cs_df.head()


Client  Lat  Long Medicine  Dosage  Assigned_Pharmacy
0  Client 000   73    95     c32e       2                  2
1  Client 001  118   125     9751       2                  8
2  Client 002  240   233     e893       2                  5
3  Client 003  198   228     2692       1                  3
4  Client 004    0    21     2692       1                  4

In [79]:
# Assign each pharmacy a list of medications that are required

# use assigned_pharmacy in cs_df to get the pharmacy name, and then get the medicine
# turn that into a counter and then into a dataframe into demand

for client in range(client_count):
    pharmacy = cs_df.loc[client, 'Assigned_Pharmacy']
    med = cs_df.loc[client, 'Medicine']
    dosage = cs_df.loc[client, 'Dosage']
    rx_df.loc[pharmacy, 'Demand'] = Counter({med: dosage})
    # needs to be fixed ^ TODO

rx_df 


ValueError: Incompatible indexer with Series